<a href="https://colab.research.google.com/github/lcylmhlcy/pytorch1.0_MostUse/blob/master/pytorch1_0_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! nvidia-smi

Thu Apr 25 12:15:31 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    31W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import collections
import os
import shutil
import tqdm

import numpy as np
import PIL.Image
import torch
import torchvision

In [0]:
print(torch.__version__)               # PyTorch version
print(torch.version.cuda)              # Corresponding CUDA version
print(torch.backends.cudnn.version())  # Corresponding cuDNN version
print(torch.cuda.get_device_name(0))   # GPU type

1.0.1.post2
10.0.130
7402
Tesla T4


In [0]:
# 判断是否有cuda支持
print(torch.cuda.is_available())

True


In [0]:
# 设置为 cuDNN benchmark 模式

# Benchmark 模式会提升计算速度，但是由于计算中有随机性，每次网络前馈结果略有差异。
torch.backends.cudnn.benchmark = True

# 如果想要避免这种结果波动，设置
torch.backends.cudnn.deterministic = True

# Note:
# 总的来说，大部分情况下，设置这个 benchmark 可以让内置的 cuDNN 的 auto-tuner 自动寻找最适合当前配置的高效算法，来达到优化运行效率的问题。
# 一般来讲，应该遵循以下准则：
# (1)如果网络的输入数据维度或类型上变化不大，设置  torch.backends.cudnn.benchmark = true  可以增加运行效率；
# (2)如果网络的输入数据在每次 iteration 都变化的话，会导致 cnDNN 每次都会去寻找一遍最优配置，这样反而会降低运行效率。

In [0]:
# 清除 GPU 存储

# 有时 Control-C 中止运行后 GPU 存储没有及时释放，需要手动清空。在 PyTorch 内部可以
torch.cuda.empty_cache()

# 或在命令行可以先使用 ps 找到程序的 PID，再使用 kill 结束该进程
ps aux | grep pythonkill -9 [pid]

# 或者直接重置没有被清空的 GPU
nvidia-smi --gpu-reset -i [gpu_id]

In [0]:
# 在训练开始时，参数的初始化是随机的，为了让每次的结果一致，我们需要设置随机种子。

# 固定GPU随机化种子
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)

# 为CPU设置随机种子
torch.manual_seed(args.seed)

In [0]:
# Tensor--------------------------------------------------------------------------------

In [0]:
a = torch.Tensor([[1,2,3],[4,5,6]])
print(a.type())
print(a.size())
print(a.dim())

torch.FloatTensor
torch.Size([2, 3])
2


In [0]:
# Set default tensor type. Float in PyTorch is much faster than double.
torch.set_default_tensor_type(torch.FloatTensor)

In [0]:
# Type convertions.
a = a.cuda() # convert Tensor a on CPU to Tensor a on GPU
a = a.cpu() # convert Tensor a on GPU to Tensor a on CPU

In [0]:
b = a.float() # convert a to FloatTensor
print(b.type())
c = a.long() # convert a to LongTensor
print(c.type())

torch.FloatTensor
torch.LongTensor


In [0]:
# torch.Tensor -> np.ndarray.
a = a.cuda()
a = a.cpu().numpy()  # convert to numpy on CPU

# np.ndarray -> torch.Tensor.
a1 = np.array([[1,2,3],[4,5,6]], dtype=np.int64)
b1 = torch.from_numpy(ndarray).float()
c1 = torch.from_numpy(ndarray.copy()).float()  # If ndarray has negative stride

In [0]:
# torch.Tensor 与 PIL.Image 转换
# PyTorch 中的张量默认采用 N×D×H×W 的顺序，并且数据范围在 [0, 1]，需要进行转置和规范化。

# torch.Tensor -> PIL.Image.
image = PIL.Image.fromarray(torch.clamp(tensor * 255, min=0, max=255).byte().permute(1, 2, 0).cpu().numpy())
image = torchvision.transforms.functional.to_pil_image(tensor)  # Equivalently way

# PIL.Image -> torch.Tensor.
tensor = torch.from_numpy(np.asarray(PIL.Image.open(path))).permute(2, 0, 1).float() / 255
tensor = torchvision.transforms.functional.to_tensor(PIL.Image.open(path))  # Equivalently way

In [0]:
# np.ndarray 与 PIL.Image 转换

# np.ndarray -> PIL.Image.
image = PIL.Image.fromarray(ndarray.astypde(np.uint8))

# PIL.Image -> np.ndarray.
ndarray = np.asarray(PIL.Image.open(path))

In [0]:
# 从只包含一个元素的张量中提取值
# 这在训练时统计 loss 的变化过程中特别有用。否则这将累积计算图，使 GPU 存储占用量越来越大。

value = tensor.item()

In [0]:
# 张量形变
# 张量形变常常需要用于将卷积层特征输入全连接层的情形。相比 torch.view，torch.reshape 可以自动处理输入张量不连续的情况。

tensor = torch.reshape(tensor, shape)

In [0]:
 # 打乱顺序

tensor = tensor[torch.randperm(tensor.size(0))]  # Shuffle the first dimension

In [0]:
# 水平翻转
# PyTorch 不支持 tensor[::-1] 这样的负步长操作，水平翻转可以用张量索引实现。

# Assume tensor has shape N*D*H*W.
tensor = tensor[:, :, :, torch.arange(tensor.size(3) - 1, -1, -1).long()]

In [0]:
# 复制张量
# 有三种复制的方式，对应不同的需求。

# Operation                 |  New/Shared memory | Still in computation graph |
tensor.clone()            # |        New         |          Yes               |
tensor.detach()           # |      Shared        |          No                |
tensor.detach.clone()()   # |        New         |          No                |

In [0]:
# 拼接张量
# 注意 torch.cat 和 torch.stack 的区别在于 torch.cat 沿着给定的维度拼接，而 torch.stack 会新增一维。
# 例如当参数是 3 个 10×5 的张量，torch.cat 的结果是 30×5 的张量，而 torch.stack 的结果是 3×10×5 的张量。

tensor = torch.cat(list_of_tensors, dim=0)
tensor = torch.stack(list_of_tensors, dim=0)

In [0]:
# 将整数标记转换成独热（one-hot）编码
# PyTorch 中的标记默认从 0 开始。

N = tensor.size(0)
one_hot = torch.zeros(N, num_classes).long()
one_hot.scatter_(dim=1, index=torch.unsqueeze(tensor, dim=1), src=torch.ones(N, num_classes).long())

In [0]:
# 得到非零/零元素

torch.nonzero(tensor)               # Index of non-zero elements
torch.nonzero(tensor == 0)          # Index of zero elements
torch.nonzero(tensor).size(0)       # Number of non-zero elements
torch.nonzero(tensor == 0).size(0)  # Number of zero elements

In [0]:
# 张量扩展

# Expand tensor of shape 64*512 to shape 64*512*7*7.
torch.reshape(tensor, (64, 512, 1, 1)).expand(64, 512, 7, 7)

In [0]:
# 矩阵乘法

# Matrix multiplication: (m*n) * (n*p) -> (m*p).
result = torch.mm(tensor1, tensor2)

# Batch matrix multiplication: (b*m*n) * (b*n*p) -> (b*m*p).
result = torch.bmm(tensor1, tensor2)

# Element-wise multiplication.
result = tensor1 * tensor2

In [0]:
# 计算两组数据之间的两两欧式距离

# X1 is of shape m*d.
X1 = torch.unsqueeze(X1, dim=1).expand(m, n, d)
# X2 is of shape n*d.
X2 = torch.unsqueeze(X2, dim=0).expand(m, n, d)
# dist is of shape m*n, where dist[i][j] = sqrt(|X1[i, :] - X[j, :]|^2)
dist = torch.sqrt(torch.sum((X1 - X2) ** 2, dim=2))

In [0]:
# ----模型定义------------------------------------------------------------------------------------------------